In [1]:
import os
import glob
import torch
import pandas as pd
import SimpleITK as sitk
from torch.utils.data import DataLoader
from torchvision.transforms import Compose
from torchvision.models.video import r3d_18, R3D_18_Weights
import numpy as np
import gc

In [ ]:
os.chdir("/media/feike/14Tdisk1/ZHY")  
print(os.getcwd())

In [ ]:
data_dir = './CT_first/images_segmentation'
mask_dir = './CT_first/masks_segmentation'
excel_path = './dl_feature.xlsx'

In [ ]:
weights = R3D_18_Weights.DEFAULT  
model = r3d_18(weights=weights) 

In [ ]:
model.stem[0] = torch.nn.Conv3d(1, 64, kernel_size=(3, 7, 7), stride=(1, 2, 2), padding=(1, 3, 3), bias=False) 
model = torch.nn.Sequential(*list(model.children())[:-1]) 

In [ ]:
def extractor(img_path, mask_path, net, use_gpu):
    img = sitk.ReadImage(img_path) 
    mask = sitk.ReadImage(mask_path)
    img_arr = sitk.GetArrayFromImage(img)
    mask_arr = sitk.GetArrayFromImage(mask)
    img_arr = img_arr.astype('float32') 
    mask_arr = mask_arr.astype('float32')
    img_arr = img_arr / 255.0
    img_arr = (img_arr - img_arr.mean()) / img_arr.std()
    mask_arr = mask_arr / mask_arr.max()
    img_arr = img_arr * mask_arr  
    img_arr = img_arr[np.newaxis, np.newaxis, ...] 
    img_tensor = torch.from_numpy(img_arr)

    if use_gpu:
        img_tensor = img_tensor.cuda()
        net = net.cuda()
    y = net(img_tensor).cpu() 
    y = torch.squeeze(y) 
    y = y.data.numpy() 
    return y

# 文件检索&匹配

In [ ]:
extensions = ['nii.gz'] 
img_files_list = [] 
mask_files_list = []

In [ ]:
for ext in extensions: 
    img_files_list.extend(glob.glob(os.path.join(data_dir, '*' + ext)))
    mask_files_list.extend(glob.glob(os.path.join(mask_dir, '*' + ext)))

In [ ]:
img_files_set = {os.path.splitext(os.path.basename(f))[0] for f in img_files_list}
mask_files_set = {os.path.splitext(os.path.basename(f))[0] for f in mask_files_list}
common_files = img_files_set.intersection(mask_files_set)

In [ ]:
img_files_list = [f for f in img_files_list if os.path.splitext(os.path.basename(f))[0] in common_files]
mask_files_list = [f for f in mask_files_list if os.path.splitext(os.path.basename(f))[0] in common_files]

# 执行提取

In [ ]:
features = [] 

In [ ]:
for img_path, mask_path in zip(img_files_list, mask_files_list): 
    print("Processing image:", img_path)
    print("Processing mask:", mask_path)
    try: # extract features
        file_name = os.path.splitext(os.path.basename(img_path))[0]
        feature = extractor(img_path, mask_path, model, use_gpu=False)
        features.append([file_name, img_path, mask_path] + feature.tolist())
        del feature
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    except Exception as e:
        print("Error processing", img_path, ":", str(e))
        continue

In [ ]:
columns = ['file_name', 'img_path', 'mask_path'] + ['feature_{}'.format(i) for i in range(len(features[0])-3)] # 定义列名
df = pd.DataFrame(features, columns=columns) # create df

/media/feike/14Tdisk1/ZHY
Processing image: ./CT_first/images_segmentation/0800428919.nii.gz
Processing mask: ./CT_first/masks_segmentation/0800428919.nii.gz
Processing image: ./CT_first/images_segmentation/7100182785.nii.gz
Processing mask: ./CT_first/masks_segmentation/7100182785.nii.gz
Processing image: ./CT_first/images_segmentation/7100191525.nii.gz
Processing mask: ./CT_first/masks_segmentation/7100191525.nii.gz
Processing image: ./CT_first/images_segmentation/7100225999.nii.gz
Processing mask: ./CT_first/masks_segmentation/7100225999.nii.gz
Processing image: ./CT_first/images_segmentation/9001247631.nii.gz
Processing mask: ./CT_first/masks_segmentation/9001247631.nii.gz
Processing image: ./CT_first/images_segmentation/9001439189.nii.gz
Processing mask: ./CT_first/masks_segmentation/9001439189.nii.gz
Processing image: ./CT_first/images_segmentation/9001471799.nii.gz
Processing mask: ./CT_first/masks_segmentation/9001471799.nii.gz
Processing image: ./CT_first/images_segmentation/9

In [ ]:
df.to_excel(excel_path, index=False)